<a href="https://colab.research.google.com/github/yuvalira/Final-Project-Adversarial-Attack-on-Tabular-Classification/blob/main/ATTACK/GBT_Attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import os
import pandas as pd
import joblib
from scipy.stats import pointbiserialr
import numpy as np
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, precision_score, recall_score, f1_score
import pickle
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from random import sample
from matplotlib.patches import FancyArrowPatch
import matplotlib.patheffects as pe
import torch
from transformers import RobertaTokenizer, RobertaForMaskedLM

# **Adversarial Attack**

### 1. Get Model and Data from Git

In [36]:
# Git clone
REPO_URL = "https://github.com/yuvalira/Final-Project-Adversarial-Attack-on-Tabular-Classification.git"
!git clone {REPO_URL}
%cd Final-Project-Adversarial-Attack-on-Tabular-Classification

# Read test dataset
test_data = pd.read_csv('data/test_data.csv')
print(f"test_data loaded: {test_data.shape}")

# Read GBT model (sklearn)
model_path = 'GBT/sklearn_GBT_model.joblib'
assert os.path.exists(model_path), f"Model file not found at {model_path}"

model_gbt = joblib.load(model_path)
print("sklearn GBT model loaded successfully.")

# Load label encoders for decoding categorical features
with open('data/label_encoders.pkl', 'rb') as f:
    encoders = pickle.load(f)
print("Label encoders loaded successfully.")

Cloning into 'Final-Project-Adversarial-Attack-on-Tabular-Classification'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 179 (delta 75), reused 77 (delta 29), pack-reused 0 (from 0)
Receiving objects: 100% (179/179), 3.65 MiB | 11.18 MiB/s, done.
Resolving deltas: 100% (75/75), done.
/content/Final-Project-Adversarial-Attack-on-Tabular-Classification/Final-Project-Adversarial-Attack-on-Tabular-Classification/Final-Project-Adversarial-Attack-on-Tabular-Classification/Final-Project-Adversarial-Attack-on-Tabular-Classification
test_data loaded: (9769, 15)
sklearn GBT model loaded successfully.
Label encoders loaded successfully.


In [37]:
# Load fine-tuned model directly from Hugging Face Hub
model_path = "ETdanR/RoBERTa_FT_adult"
tokenizer = RobertaTokenizer.from_pretrained(model_path)
model = RobertaForMaskedLM.from_pretrained(model_path)
model.eval()

# Define mask token info
mask_token = tokenizer.mask_token
mask_token_id = tokenizer.mask_token_id

# Token IDs for income prediction (example IDs from your snippet)
greater_than_id = 9312  # "Greater"
less_than_id = 10862    # "Less"

In [47]:
def predict_LM(df):
    sentences = [
        f"age: {row['age']}, workclass: {row['workclass']}, education: {row['education']}, "
        f"educational-num: {row['educational-num']}, marital-status: {row['marital-status']}, "
        f"occupation: {row['occupation']}, relationship: {row['relationship']}, race: {row['race']}, "
        f"gender: {row['gender']}, capital-gain: {row['capital-gain']}, capital-loss: {row['capital-loss']}, "
        f"hours-per-week: {row['hours-per-week']}, native-country: {row['native-country']}, "
        f"income: {mask_token} than 50k"
        for _, row in df.iterrows()
    ]

    encoded = tokenizer(
        sentences,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=128
    )

    with torch.no_grad():
        outputs = model(**encoded)
        logits = outputs.logits

    mask_positions = (encoded['input_ids'] == mask_token_id).nonzero(as_tuple=False)
    predictions = torch.argmax(logits, dim=-1)

    results = []
    for i in range(len(df)):
        mask_index = mask_positions[mask_positions[:, 0] == i][0, 1]
        predicted_token_id = predictions[i, mask_index].item()

        if predicted_token_id == greater_than_id:
            results.append(1)
        elif predicted_token_id == less_than_id:
            results.append(0)
        else:
            results.append(-1)

    # Convert to numpy array with shape (batch_size,)
    results_np = np.array(results, dtype=int)

    print("predict called")
    return results_np

### 2. Prepare Data-Frames

Create Correct Predictions and Incorrect Predictions datasets

In [39]:
TARGET_COLUMN_NAME = 'income'

# Create a copy of test_data
test_data_attack = test_data.copy()

# Prepare test features
exclude_columns = [TARGET_COLUMN_NAME, 'fnlwgt']
X_test = test_data.drop(columns=exclude_columns)
y_true = test_data[TARGET_COLUMN_NAME].values

# Predict class labels (0/1)
y_pred = model_gbt.predict(X_test)

# Create a new column for prediction correctness
test_data_attack['correct_prediction'] = (y_true == y_pred)

# Create datasets for correct and incorrect predictions
correct_predictions_df = test_data_attack[test_data_attack['correct_prediction'] == True].drop(columns=['correct_prediction'])
incorrect_predictions_df = test_data_attack[test_data_attack['correct_prediction'] == False].drop(columns=['correct_prediction'])

print(f"Correct predictions shape : {correct_predictions_df.shape}")
print(f"Incorrect predictions shape : {incorrect_predictions_df.shape}")


Correct predictions shape : (8572, 15)
Incorrect predictions shape : (1197, 15)


### 3. Create Correlation Dictionary

We compute a correlation dictionary to guide and constrain the adversarial attack process.  
For each feature and possible value, we calculate the correlation between the presence of that value and the target label.

The correlation dictionary serves three key purposes:
1. **Feature Filtering:** Only features with correlation above a defined threshold are considered for attack.
2. **Feature Prioritization:** Features are sorted by correlation strength to guide the attack toward the most influential attributes first.
3. **Categorical Attack Control:** For categorical features, value changes are limited to neighboring values based on their position in the correlation dictionary, avoiding random or unrealistic jumps between categories.

In [40]:
def create_correlation_dict(df, target_column='income'):
    """
    Create a dictionary of correlation values between feature values and the target class.
    Used for driving adversarial attack modifications.
    """
    correlation_dict = {}
    target = df[target_column]
    features = df.drop(columns=[target_column, 'fnlwgt'])

    for col in features.columns:
        value_corrs = {}

        # Categorical or numerical feature
        if features[col].dtype == 'object' or features[col].dtype.name == 'category':
            unique_vals = features[col].unique()
        else:
            unique_vals = np.unique(features[col].values)

        for val in unique_vals:
            binary_vec = features[col].apply(lambda x: 1 if x == val else 0)
            try:
                corr, _ = pointbiserialr(binary_vec, target)
            except Exception:
                corr = 0

            # Clean numpy types
            if isinstance(val, (np.integer, np.int64)):
                val = int(val)

            value_corrs[val] = corr if not np.isnan(corr) else 0

        # Sort values by correlation descending
        correlation_dict[col] = dict(sorted(value_corrs.items(), key=lambda x: x[1], reverse=True))

    return correlation_dict

In [41]:
correlation_dict = create_correlation_dict(correct_predictions_df, target_column="income")

# Print correlation dict
for feature, val_corrs in correlation_dict.items():
    print(f"\nFeature: {feature}")
    for val, corr in val_corrs.items():
        # If feature is categorical and encoder exists → decode value
        if feature in encoders:
            val = encoders[feature].inverse_transform([int(val)])[0]
        print(f"    {val}: {corr:.4f}")


Feature: age
    46: 0.0828
    47: 0.0791
    45: 0.0771
    42: 0.0736
    50: 0.0650
    51: 0.0602
    48: 0.0591
    44: 0.0572
    38: 0.0552
    52: 0.0537
    43: 0.0480
    41: 0.0478
    39: 0.0395
    53: 0.0371
    49: 0.0353
    40: 0.0349
    37: 0.0331
    54: 0.0311
    58: 0.0310
    57: 0.0280
    55: 0.0255
    83: 0.0229
    60: 0.0168
    36: 0.0166
    59: 0.0157
    79: 0.0154
    56: 0.0150
    61: 0.0140
    32: 0.0091
    62: 0.0084
    73: 0.0020
    63: 0.0018
    78: 0.0012
    80: 0.0012
    65: -0.0003
    34: -0.0020
    70: -0.0020
    69: -0.0029
    71: -0.0030
    67: -0.0041
    81: -0.0045
    64: -0.0051
    88: -0.0051
    84: -0.0072
    76: -0.0072
    90: -0.0084
    66: -0.0093
    82: -0.0102
    35: -0.0126
    68: -0.0139
    75: -0.0161
    77: -0.0161
    74: -0.0176
    72: -0.0216
    33: -0.0258
    30: -0.0264
    31: -0.0297
    29: -0.0409
    17: -0.0534
    28: -0.0537
    27: -0.0647
    24: -0.0656
    18: -0.0666
    25: -0.0

### 4. Adversarial Attack Generation

This function generates adversarial examples by applying controlled perturbations to the input samples.  
The attack follows a structured approach to efficiently search for minimal changes that will flip the model's prediction:

1. **Input Selection:** Only correctly predicted samples are selected for the attack.
2. **Feature Selection:** Features are prioritized and filtered based on the correlation dictionary.
3. **Attack Loop:** For each sample:
   - For **numeric features**, the value is perturbed by adding or subtracting a small pre-defined delta.
   - For **categorical features**, the current value is replaced by neighboring values in the correlation dictionary (not random replacements).
   - After each change, the model is queried. If the prediction flips, the attack stops for that sample.

This method ensures that the attack remains realistic and focused on high-impact feature manipulations, avoiding excessive or implausible modifications.

In [42]:
def generate_adversarial_attack(
        df_test, model_gbt, correlation_dict, modelArch,
        max_features_changed=3,
        correlation_threshold=0.05, max_attempts=10):
    # Declare bounds for numeric features
    numeric_bounded_features = {
        'age': 5,
        'hours-per-week': 5,
        'capital-gain': 1000,
        'capital-loss': 1000,
        'educational-num': 2,
    }

    adv_samples = []
    success_count = 0
    df_types = df_test.dtypes.to_dict()

    df_test = df_test.sample(n=32, random_state=42)

    # Prepare test set
    exclude_columns = [TARGET_COLUMN_NAME, 'fnlwgt']
    X_test = df_test.drop(columns=exclude_columns)
    y_true = df_test[TARGET_COLUMN_NAME].values
    y_pred_orig = model_gbt.predict(X_test) if modelArch == "GBT" else predict_LM(X_test)

    # Prioritize features by correlation
    priorities = []
    for feature in correlation_dict.keys():
        if feature == TARGET_COLUMN_NAME or feature not in df_test.columns:
            continue
        values = correlation_dict[feature].values()
        max_corr = max(abs(v) for v in values if v is not None)
        if max_corr >= correlation_threshold:
            priorities.append((feature, max_corr))

    priorities.sort(key=lambda x: x[1], reverse=True)
    ordered_features = [f[0] for f in priorities]

    for idx in range(len(df_test)):
        original_input = df_test.iloc[[idx]]
        original_label = original_input[TARGET_COLUMN_NAME].values[0]
        current_sample = original_input.iloc[0].copy()
        changed_features = set()
        success = False
        attempts = 0

        while len(changed_features) < max_features_changed and attempts < max_attempts:
            attempts += 1
            current_X = pd.DataFrame([current_sample.drop(labels=[TARGET_COLUMN_NAME, 'fnlwgt'])])
            current_pred = model_gbt.predict(current_X)[0] if modelArch == "GBT" else predict_LM(current_X)[0]

            if current_pred != original_label:
                success = True
                break

            candidate_samples = []
            candidate_features = []

            for feature in ordered_features:
                if feature in changed_features:
                    continue

                current_value = current_sample[feature]

                # Numeric feature
                if feature in numeric_bounded_features:
                    delta = numeric_bounded_features[feature]
                    try:
                        current_value = float(current_value)
                        for new_val in [current_value + delta, max(0, current_value - delta)]:
                            trial_sample = current_sample.copy()
                            trial_sample[feature] = new_val
                            candidate_samples.append(trial_sample)
                            candidate_features.append(feature)
                    except:
                        continue

                # Categorical feature
                else:
                    possible_vals = list(correlation_dict[feature].keys())
                    current_value_int = int(current_value) if isinstance(current_value,
                                                                         (np.integer, np.int64)) else current_value
                    try:
                        idx_val = possible_vals.index(current_value_int)
                    except ValueError:
                        continue

                    neighbors = []
                    if idx_val - 2 >= 0:
                        neighbors.append(possible_vals[idx_val - 2])
                    if idx_val + 2 < len(possible_vals):
                        neighbors.append(possible_vals[idx_val + 2])
                    if not neighbors and len(possible_vals) > 1:
                        if idx_val > 0:
                            neighbors.append(possible_vals[idx_val - 1])
                        if idx_val < len(possible_vals) - 1:
                            neighbors.append(possible_vals[idx_val + 1])

                    for neighbor_val in neighbors:
                        trial_sample = current_sample.copy()
                        trial_sample[feature] = neighbor_val
                        candidate_samples.append(trial_sample)
                        candidate_features.append(feature)

                if len(candidate_samples) >= 20:
                    break

            if not candidate_samples:
                break

            batch_X = pd.DataFrame(candidate_samples)[
                [col for col in current_sample.index if col not in [TARGET_COLUMN_NAME, 'fnlwgt']]]
            batch_preds = model_gbt.predict(batch_X) if modelArch == "GBT" else predict_LM(batch_X)

            for i, pred in enumerate(batch_preds):
                if int(pred) != original_label:
                    current_sample = candidate_samples[i]
                    changed_features.add(candidate_features[i])
                    break

        adv_samples.append(current_sample.astype(object))
        if success:
            success_count += 1

    adv_df = pd.DataFrame(adv_samples).astype(df_types)

    # Evaluation
    adv_X = adv_df.drop(columns=[TARGET_COLUMN_NAME, 'fnlwgt'])
    y_pred_adv = model_gbt.predict(adv_X) if modelArch == "GBT" else predict_LM(adv_X)

    flipped = sum(y_pred_orig != y_pred_adv)
    print(f"\n Attack success: {success_count}/{len(df_test)} ({(success_count / len(df_test)) * 100:.2f}%)")
    print(f" Original accuracy: {accuracy_score(y_true, y_pred_orig):.2%}")
    print(f" Adversarial accuracy: {accuracy_score(y_true, y_pred_adv):.2%}")

    return adv_df

Now, we apply the adversarial attack on the set of correctly predicted test samples from the Gradient Boosted Trees (GBT) model.  

This step generates the adversarial dataset (`adv_test_df`) used for further evaluation and analysis.


In [48]:
# Select subset of samples to attack
samples_to_attack = correct_predictions_df.copy()

# Run adversarial attack
adv_test_df = generate_adversarial_attack(
    df_test=samples_to_attack,
    model_gbt=model_gbt,
    correlation_dict=correlation_dict,
    modelArch = "LM" # "GBT" or "LM"
)

print(f"\n Attack finished. Adversarial test set shape: {adv_test_df.shape}")

predict called
predict called
predict called
predict called
predict called
predict called
predict called
predict called
predict called
predict called


KeyboardInterrupt: 

### 5. Attack Evaluation

In [44]:
def analyze_attack_changes(original_df, adversarial_df, label_column='income', verbose=True, max_samples_to_print=10):
    """
    Analyze feature changes between original and adversarial samples.
    Returns: list of differences per sample, and feature change counts.
    Only prints up to max_samples_to_print changed samples for readability.
    """
    feature_names = [f for f in original_df.columns if f not in [label_column, 'fnlwgt']]
    change_counts = {feature: 0 for feature in feature_names}
    total_samples = len(original_df)

    all_diffs = []
    printed_samples = 0

    for idx in range(total_samples):
        orig_row = original_df.iloc[idx]
        adv_row = adversarial_df.iloc[idx]
        diffs = {}
        for feature in feature_names:
            if orig_row[feature] != adv_row[feature]:
                old_val = orig_row[feature]
                new_val = adv_row[feature]

                # decode categorical values back to original labels
                if feature in encoders:
                    try:
                        old_val = encoders[feature].inverse_transform([int(old_val)])[0]
                        new_val = encoders[feature].inverse_transform([int(new_val)])[0]
                    except Exception:
                        pass  # in case of any weird value, just keep as is

                diffs[feature] = (old_val, new_val)
                change_counts[feature] += 1

        all_diffs.append(diffs)

        if verbose and diffs and printed_samples < max_samples_to_print:
            print(f"\nSample {idx}: {len(diffs)} feature(s) changed:")
            for feat, (old, new) in diffs.items():
                print(f"  {feat}: {old} → {new}")
            printed_samples += 1

    # Summary
    print("\n--- Global Feature Change Statistics ---")
    for feature, count in change_counts.items():
        print(f"{feature}: changed in {count}/{total_samples} samples ({(count/total_samples)*100:.2f}%)")

    print("\n Attack analysis completed.")
    return all_diffs, change_counts

In [45]:
all_diffs, change_counts = analyze_attack_changes(
    original_df=samples_to_attack,
    adversarial_df=adv_test_df,
    label_column='income'
)


Sample 0: 7 feature(s) changed:
  age: 56 → 52
  education: HS-grad → 10th
  educational-num: 9 → 6
  marital-status: Divorced → Never-married
  occupation: Other-service → Machine-op-inspct
  relationship: Unmarried → Not-in-family
  gender: Female → Male

Sample 1: 6 feature(s) changed:
  age: 25 → 49
  workclass: Private → Self-emp-not-inc
  marital-status: Married-civ-spouse → Divorced
  occupation: Transport-moving → Sales
  relationship: Own-child → Not-in-family
  race: Other → White

Sample 2: 5 feature(s) changed:
  age: 43 → 25
  workclass: Private → Local-gov
  marital-status: Divorced → Never-married
  relationship: Not-in-family → Own-child
  capital-gain: 14344 → 0

Sample 3: 6 feature(s) changed:
  age: 32 → 22
  education: HS-grad → 11th
  educational-num: 9 → 7
  marital-status: Married-civ-spouse → Never-married
  occupation: Transport-moving → Adm-clerical
  relationship: Husband → Not-in-family

Sample 4: 8 feature(s) changed:
  age: 20 → 25
  education: HS-grad → 

IndexError: single positional indexer is out-of-bounds

**Model Robustness Comparison**

Now, we will evaluate the model on both the original test set and the full attacked test set.  


In [ ]:
# Predict on original test set (before attack)
X_original = test_data.drop(columns=['income', 'fnlwgt'])
y_original = test_data['income']

y_pred_original = model_gbt.predict(X_original)

# Combine attacked + incorrect samples to create full attacked dataset
full_attacked_test_data = pd.concat([adv_test_df, incorrect_predictions_df], axis=0).reset_index(drop=True)

X_attacked = full_attacked_test_data.drop(columns=['income', 'fnlwgt'])
y_attacked = full_attacked_test_data['income']

y_pred_attacked = model_gbt.predict(X_attacked)

# Calculate metrics for original test set
original_accuracy = accuracy_score(y_original, y_pred_original)
original_precision = precision_score(y_original, y_pred_original)
original_recall = recall_score(y_original, y_pred_original)
original_f1 = f1_score(y_original, y_pred_original)

# Calculate metrics for attacked test set
attacked_accuracy = accuracy_score(y_attacked, y_pred_attacked)
attacked_precision = precision_score(y_attacked, y_pred_attacked)
attacked_recall = recall_score(y_attacked, y_pred_attacked)
attacked_f1 = f1_score(y_attacked, y_pred_attacked)

# Print comparison
print("\n Model Performance Comparison:")
print(f" Original Accuracy: {original_accuracy:.4f} | Attacked Accuracy: {attacked_accuracy:.4f}")
print(f" Original Precision: {original_precision:.4f} | Attacked Precision: {attacked_precision:.4f}")
print(f" Original Recall: {original_recall:.4f} | Attacked Recall: {attacked_recall:.4f}")
print(f" Original F1-Score: {original_f1:.4f} | Attacked F1-Score: {attacked_f1:.4f}")

# Accuracy drop
drop = original_accuracy - attacked_accuracy
print(f"\n Accuracy drop due to attack: {drop:.4f} ({drop*100:.2f}% drop)")


**ROC Curve Comparison Before and After Attack**

We compare the Receiver Operating Characteristic (ROC) curves of the model on the original test set and after the adversarial attack.  
A lower ROC curve and AUC after attack indicates degradation of the model's classification ability.

In [ ]:
# Original test set
X_original = test_data.drop(columns=['income', 'fnlwgt'])
y_original = test_data['income']
y_score_original = model_gbt.predict_proba(X_original)[:, 1]
auc_original = roc_auc_score(y_original, y_score_original)
fpr_original, tpr_original, _ = roc_curve(y_original, y_score_original)

# Attacked test set
X_attacked = full_attacked_test_data.drop(columns=['income', 'fnlwgt'])
y_attacked = full_attacked_test_data['income']
y_score_attacked = model_gbt.predict_proba(X_attacked)[:, 1]
auc_attacked = roc_auc_score(y_attacked, y_score_attacked)
fpr_attacked, tpr_attacked, _ = roc_curve(y_attacked, y_score_attacked)

# Plot both curves
plt.figure(figsize=(8,6))
plt.plot(fpr_original, tpr_original, label=f"Original Test Set (AUC = {auc_original:.4f})", linewidth=2)
plt.plot(fpr_attacked, tpr_attacked, label=f"Attacked Test Set (AUC = {auc_attacked:.4f})", linewidth=2, linestyle='--')
plt.plot([0, 1], [0, 1], 'k--', label="Random Classifier (AUC = 0.5)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve: Before vs After Adversarial Attack")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()


The ROC curve comparison shows a significant reduction in model performance under adversarial attack.  
The AUC dropped from 0.9299 to 0.8351, demonstrating that the structured attack succeeded in reducing the model's ability to distinguish between income classes.

**PCA**

In this analysis, we applied PCA (Principal Component Analysis) to two datasets: one before an adversarial attack and one after.

The purpose of PCA is to project high-dimensional data into a lower-dimensional space (2D) while preserving as much variance (information) as possible.
This allows us to:


*   Visually compare the distribution of data before and after the attack.
*   Observe whether the attack caused meaningful changes in the data structure.
*   Identify if there is a visual separation between the two datasets, which may indicate that the attack significantly altered the features.

In [ ]:
# Map numeric labels to class names
class_map = {
    0: '<= 50k',
    1: '> 50k'
}

# Define colors for each class name
custom_colors = {
    '<= 50k': 'lightsteelblue',  # light blue
    '> 50k': 'dimgrey'  # gray
}

# Preprocess function
def preprocess(df, label_column='income'):
    raw_labels = df[label_column].values
    labels = np.vectorize(class_map.get)(raw_labels)  # convert to class names
    features = df.drop(columns=[label_column])
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    return features_scaled, labels

# Preprocess datasets
X_original, y_original = preprocess(test_data, label_column='income')
X_attacked, y_attacked = preprocess(full_attacked_test_data, label_column='income')

# PCA
pca = PCA(n_components=2)
X_original_pca = pca.fit_transform(X_original)
X_attacked_pca = pca.transform(X_attacked)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Original
for label in np.unique(y_original):
    idx = y_original == label
    axes[0].scatter(X_original_pca[idx, 0], X_original_pca[idx, 1],
                    c=custom_colors[label], label=label, alpha=0.6)
axes[0].set_title('Original Test Data')
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')
axes[0].legend(title='Income')

# Attacked
for label in np.unique(y_attacked):
    idx = y_attacked == label
    axes[1].scatter(X_attacked_pca[idx, 0], X_attacked_pca[idx, 1],
                    c=custom_colors[label], label=label, alpha=0.6)
axes[1].set_title('Attacked Test Data')
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC2')
axes[1].legend(title='Income')

plt.tight_layout()
plt.show()

In [ ]:
# Class name map
class_map = {0: '<= 50K', 1: '> 50K'}

# Soft transition colors
transition_colors = {
    (0, 1): '#FF6F61',  # coral rose
    (1, 0): '#6BA292'   # eucalyptus green
}

# Background colors
background_colors = {
    0: 'lightsteelblue',
    1: 'dimgrey'
}

# Preprocess
def preprocess(df, label_column='income'):
    labels = df[label_column].values.astype(int)
    features = df.drop(columns=[label_column])
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    return features_scaled, labels

# Prepare data
X_original, y_original = preprocess(test_data)
X_attacked, y_attacked = preprocess(full_attacked_test_data)

# Changed samples
changed_idx = np.where(y_original != y_attacked)[0]
X_orig_changed = X_original[changed_idx]
X_att_changed = X_attacked[changed_idx]
y_from = y_original[changed_idx]
y_to = y_attacked[changed_idx]

# Select 5 from each transition
transition_01 = np.where((y_from == 0) & (y_to == 1))[0]
transition_10 = np.where((y_from == 1) & (y_to == 0))[0]
selected_01 = sample(list(transition_01), min(5, len(transition_01)))
selected_10 = sample(list(transition_10), min(5, len(transition_10)))
selected_idx = selected_01 + selected_10

X_orig_sel = X_orig_changed[selected_idx]
X_att_sel = X_att_changed[selected_idx]
y_from_sel = y_from[selected_idx]
y_to_sel = y_to[selected_idx]

# PCA
pca = PCA(n_components=2)
X_original_pca = pca.fit_transform(X_original)
X_orig_pca_sel = pca.transform(X_orig_sel)
X_att_pca_sel = pca.transform(X_att_sel)

# Plot
fig, ax = plt.subplots(figsize=(10, 7))

# Background
for label in np.unique(y_original):
    idx = y_original == label
    ax.scatter(X_original_pca[idx, 0], X_original_pca[idx, 1],
               c=background_colors[label], label=f"{class_map[label]} (background)",
               alpha=0.2, s=30)

# Arrows + points
for i in range(len(selected_idx)):
    x0, y0 = X_orig_pca_sel[i]
    x1, y1 = X_att_pca_sel[i]
    transition = (y_from_sel[i], y_to_sel[i])
    color = transition_colors.get(transition, 'gray')

    # Fancy arrow with outline
    arrow = FancyArrowPatch((x0, y0), (x1, y1),
                            arrowstyle='->',
                            color=color,
                            linewidth=1.5,
                            mutation_scale=14,
                            alpha=0.85,
                            path_effects=[
                                pe.Stroke(linewidth=1.5, foreground='black'),
                                pe.Normal()
                            ])
    ax.add_patch(arrow)

    # End point
    ax.scatter(x1, y1, c=color, s=80, edgecolor='black', marker='o')

# Final touches
ax.set_title('PCA with Sample of Adversarial Shifts', fontsize=14)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.legend(title='income')
plt.tight_layout()
plt.show()